# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

The environments corresponding to both versions of the environment are already saved in the Workspace and can be accessed at the file paths provided below.  

Please select one of the two options below for loading the environment.

In [ ]:
from unityagents import UnityEnvironment
import numpy as np
import copy
from collections import namedtuple, deque
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# For visualiztion for trained model in Windows PC
env = UnityEnvironment(file_name='Reacher_Windows_x86_64/Reacher.exe')


# select this option to load version 1 (with a single agent) of the environment
# env = UnityEnvironment(file_name='/data/Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64')

# select this option to load version 2 (with 20 agents) of the environment
# env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64')

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [  0.00000000e+00  -4.00000000e+00   0.00000000e+00   1.00000000e+00
  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00  -1.00000000e+01   0.00000000e+00
   1.00000000e+00  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   5.75471878e+00  -1.00000000e+00
   5.55726671e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
  -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [5]:
env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: 0.0


When finished, you can close the environment.

In [6]:
# env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 

### 5. My DDPG

### 5.1 Actor and Critic

In [7]:
class Actor(nn.Module):
    def __init__(self, state_size, action_size, param_file=None):
        super().__init__()
        self.fc1 = nn.Linear(state_size, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, action_size)
        if param_file:
            self.load_params(param_file)
        else:
            self._reset_params()

    def _reset_params(self):
        nn.init.kaiming_normal_(self.fc1.weight)
        nn.init.constant_(self.fc1.bias, 0)
        nn.init.kaiming_normal_(self.fc2.weight)
        nn.init.constant_(self.fc2.bias, 0)
        nn.init.xavier_normal_(self.fc3.weight)
        nn.init.constant_(self.fc3.bias, 0)

    def load_params(self, param_file):
        self.load_state_dict(torch.load(param_file))
        
    def forward(self, state):
        x = state
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return torch.tanh(self.fc3(x))


class Critic(nn.Module):
    def __init__(self, state_size, action_size, param_file=None):
        super().__init__()
        self.fc1 = nn.Linear(state_size, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512 + action_size, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 1)
        if param_file:
            self.load_params(param_file)
        else:
            self._reset_params()

    def _reset_params(self):
        nn.init.kaiming_normal_(self.fc1.weight)
        nn.init.constant_(self.fc1.bias, 0)
        nn.init.kaiming_normal_(self.fc2.weight)
        nn.init.constant_(self.fc2.bias, 0)
        nn.init.xavier_normal_(self.fc3.weight)
        nn.init.constant_(self.fc3.bias, 0)
    
    def load_params(self, param_file):
        self.load_state_dict(torch.load(param_file))

    def forward(self, state, action):
        x = F.relu(self.bn1(self.fc1(state)))
        x = torch.cat((x, action), dim=1)
        x = F.relu(self.bn2(self.fc2(x)))
        return self.fc3(x)

### 5.2 DDPG

In [8]:
import numpy as np
import torch
import torch.nn.functional as F
import copy
from collections import deque, namedtuple
import random

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

class ReplayBuffer():
    def __init__(self, sample_size, max_len=int(1e5)):
        self.buffer = deque(maxlen=max_len)
        self.sample_size = sample_size
        self.experience = namedtuple('experience', ('state', 'action', 'reward', 'next_state', 'done'))

    def add(self, states, actions, rewards, next_states, dones):
        exps = [self.experience(s, a, r, ns, d)
                for s, a, r, ns, d in zip(states, actions, rewards, next_states, dones)]
        self.buffer.extend(exps)

    def sample(self):
        exps = random.sample(self.buffer, self.sample_size)

        states = torch.from_numpy(np.vstack([e.state for e in exps if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in exps if e is not None])).float().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in exps if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in exps if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in exps if e is not None]).astype(np.uint8)).float().to(device)

        return states, actions, rewards, next_states, dones

    def __len__(self):
        return len(self.buffer)


class Agent():
    def __init__(self, state_size, action_size, batch_size=128, 
                 actor_alpha=1e-3, critic_alpha=1e-3, gamma=0.99, tau=1e-3, 
                 device=device, actor_file=None, critic_file=None):
        # hyperparameters
        self.gamma = gamma
        self.tau = tau
        self.noise_decay = 1 + 5e-6
        # env info
        self.state_size = state_size
        self.action_size = action_size
        # replay buffer
        self.buffer = ReplayBuffer(sample_size=batch_size)
        # actor-critic
        self.actor_main = Actor(state_size, action_size, param_file=actor_file).to(device)
        self.critic_main = Critic(state_size, action_size, param_file=critic_file).to(device)
        # target actor-critic
        self.actor_target = copy.deepcopy(self.actor_main)
        self.critic_target = copy.deepcopy(self.critic_main)
        # optimizer
        self.actor_optimizer = torch.optim.Adam(self.actor_main.parameters(), lr=actor_alpha)
        self.critic_optimizer = torch.optim.Adam(self.critic_main.parameters(), lr=critic_alpha)

    def act(self, states):
        states = torch.from_numpy(states).float().to(device)
        # add noise to parameters
        saved_params = []
        for param in self.actor_main.parameters():
            saved_params.append(copy.deepcopy(param))
            param = param + torch.normal(mean=0.0, std=torch.ones_like(param) / (10 * self.noise_decay))

        self.noise_decay *= 1 + 5e-6
        
        self.actor_main.eval()
        with torch.no_grad():
           actions = self.actor_main(states).cpu().numpy()
        self.actor_main.train()
        # restore parameters
        for param, saved_param in zip(self.actor_main.parameters(), saved_params):
            param = saved_param

        return np.clip(actions, -1, 1)

    def step(self, states, actions, rewards, next_states, dones):
        self.buffer.add(states, actions, rewards, next_states, dones)

        if len(self.buffer) > self.buffer.sample_size + 100:
            self._learn()

    def _learn(self):
        states, actions, rewards, next_states, dones = self.buffer.sample()
        targets = rewards + (1 - dones) * self.gamma * self.critic_target(next_states, self.actor_target(next_states)).detach()
        critic_loss = F.mse_loss(self.critic_main(states, actions), targets)
        actor_loss = -self.critic_main(states, self.actor_main(states)).mean()
        # update critic
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        self.critic_optimizer.step()
        # update actor
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
        # update the target networks
        self._moving_average(self.actor_main, self.actor_target)
        self._moving_average(self.critic_main, self.critic_target)

    def _moving_average(self, main, target):
        for target_param, main_param in zip(target.parameters(), main.parameters()):
            target_param.data.copy_(self.tau * main_param.data + (1.0 - self.tau) * target_param.data)

### 5.3 Training DDPG

In [ ]:
num_episodes = 1000
agent = Agent(state_size, action_size)
# agent.actor_main.load_state_dict(torch.load('actor_checkpoint.pth'))
# agent.critic_main.load_state_dict(torch.load('critic_checkpoint.pth'))

avg_scores = deque(maxlen=100)
for eps in range(1, num_episodes+1):
    env_info = env.reset(train_mode=True)[brain_name]          
    states = env_info.vector_observations
    scores = np.zeros(num_agents)        
    best_scores = scores
    while True:
        actions = agent.act(states)
        env_info = env.step(actions)[brain_name]
        next_states = env_info.vector_observations         
        rewards = env_info.rewards                         
        dones = env_info.local_done
        agent.step(states, actions, rewards, next_states, dones)
        scores += env_info.rewards                         
        states = next_states                               
        if np.any(dones):                                  
            break
    avg_scores.append(scores)
    if scores[0] > best_scores[0]:
        best_scores = scores
    avg_score = np.mean(avg_scores)
    print('\rEpisode {},\tScores: {:.2f},\t Average scores: {:.2f}'.format(eps, scores[0], avg_score), end='')
    if eps % 100 == 0:
        print('\rAverage score over 100 episodes: {:.2f}\tBest score: {}'.format(np.mean(avg_scores), best_scores[0]))
    torch.save(agent.actor_main.state_dict(), 'actor_checkpoint.pth')
    torch.save(agent.critic_main.state_dict(), 'critic_checkpoint.pth')
    if eps > 100 and avg_score > 30:
        print('\rTarget achieved. Average score over 100 episodes: {}'.format(avg_score))
        break

Average score over 100 episodes: 8.99	Best score: 25.419999431818724
Average score over 100 episodes: 28.31	Best score: 37.83999915421009
Episode 226,	Scores: 31.51,	 Average scores: 27.55

### 5.4 Visualizing trained model

In [ ]:
num_episodes = 1000
agent = Agent(state_size, action_size)
agent.actor_main.load_state_dict(torch.load('actor_checkpoint.pth'))
agent.critic_main.load_state_dict(torch.load('critic_checkpoint.pth'))

avg_scores = deque(maxlen=100)
for eps in range(1, num_episodes+1):
    env_info = env.reset(train_mode=True)[brain_name]          
    states = env_info.vector_observations
    scores = np.zeros(num_agents)        
    best_scores = scores
    while True:
        actions = agent.act(states)
        env_info = env.step(actions)[brain_name]
        next_states = env_info.vector_observations         
        rewards = env_info.rewards                         
        dones = env_info.local_done
#         agent.step(states, actions, rewards, next_states, dones)
        scores += env_info.rewards                         
        states = next_states                               
        if np.any(dones):                                  
            break
    avg_scores.append(scores)
    if scores[0] > best_scores[0]:
        best_scores = scores
    avg_score = np.mean(avg_scores)
    print('\rEpisode {},\tScores: {:.2f},\t Average scores: {:.2f}'.format(eps, scores[0], avg_score), end='')
    if eps % 100 == 0:
        print('\rAverage score over 100 episodes: {:.2f}\tBest score: {}'.format(np.mean(avg_scores), best_scores[0]))
#     torch.save(agent.actor_main.state_dict(), 'actor_checkpoint.pth')
#     torch.save(agent.critic_main.state_dict(), 'critic_checkpoint.pth')
    if eps > 100 and avg_score > 30:
        print('\rTarget achieved. Average score over 100 episodes: {}'.format(avg_score))
        break

In [ ]:
env.close()